## Importing necessary libraries

- **pandas** - Used for data loading and storing in dataframes
- **sqlalchemy** - Library for leveraging SQL databases, used here due to its efficiency in handling small amounts of data and simplicity
- **geopy** - 
- **folium** - 

In [37]:
import pandas as pd
import sqlite3
from geopy.distance import geodesic
import folium

## Data Loading

Here we are using pandas `.read_csv()` method for reading data from csv file and storing it into pandas `DataFrame` which is clear and similar to SQL format

In [38]:
# Reading csv file and loading into DataFrame
data = pd.read_csv('points.csv')

# Let's see the loaded data
data.head(3)

,name,coordinates,height,date_added,metadata
0,Exatel HQ,"52.2322, 21.0083",0.115,2024-05-06T09:00:00,sector=telecommunications;status=headquarters
1,Wrocław IT Center,"51.1079, 17.0385",0.142,2024-05-06T09:15:00,sector=IT services;status=data center
2,Kraków Cyber Hub,"50.0647, 19.9450",0.206,2024-05-06T09:30:00,sector=cybersecurity;status=training facility


## Setting up Database

In [39]:
# Establishing the connection to database
conn = sqlite3.connect('points.db')

# Setting the control structure using .cursor() to enable executing queries in the future
cursor = conn.cursor()

# Query for dropping table if exist
drop_query = '''
DROP TABLE IF EXISTS points
'''

# Executing drop table query
cursor.execute(drop_query)

# Query for creating table points if not exists already with columns same as in csv file
create_query = '''
CREATE TABLE points (
    id INT PRIMARY KEY,
    name TEXT,
    latitude REAL,
    longitude REAL,
    height REAL,
    date_added TEXT,
    metadata TEXT
)
'''

# Executing create table query
cursor.execute(create_query)

# Commiting changes
conn.commit()

## Useful functions


### Convert coordinate function

In [40]:
# Converting coordinate function
def convert_coordinate(coord: str) -> float:
    """
    Convert various geographical coordinate formats to decimal degrees.
    
    Args:
        coord (str): Coordinate presented in different formats:
            1. Degrees with decimal (e.g., 34.1234°)
            2. Degrees and minutes with decimal (e.g., 34° 12.345')
            3. Degrees, minutes, and seconds (e.g., 34° 12' 34")
            4. Includes direction letters (N, E, S, W) (e.g., 34° 5' 15.22" W)
    
    Returns:
        converted_coord (float) : Converted coordinate in decimal degrees format (1).
    """
    
    # Replacing whitespaces for easier string manipulation
    coord = coord.replace(" ", "")
    
    # Initialize components
    degrees = 0
    minutes = 0
    seconds = 0
    
    # Extracting degrees
    if '°' in coord:
        degrees = float(coord[: coord.index('°')])
    # Extracting minutes
    if "'" in coord:
        minutes = float(coord[coord.index("°") + 1 : coord.index("'")])
    # Extracting seconds
    if '"' in coord:
        seconds = float(coord[coord.index("'") + 1 : coord.index('"')])
        
    # Calcualting result
    minutes += seconds / 60
    converted_coord = degrees + minutes / 60
    
    # Handling the case with NESW letters
    if 'S' in coord or 'W' in coord:
        converted_coord *= -1
    
    # Returning result
    return converted_coord


### Get perimeter function

In [ ]:
# Calculating perimeter function
def calculate_perimeter(data):
    perimeter = 0
    points = list(data['Converted Coordinates'])
    for i in range(len(points)-1):
        perimeter += geodesic(points[i], points[i+1]).km
    perimeter += geodesic(points[-1], points[0]).km
    return perimeter

## Data Normalization

I decided to divide coordinates into longitude and latitude, all in the same single format (degrees with decimal points).

In [41]:
# Splitting coordinates column into its latitude and longitude
data[['latitude', 'longitude']] = data['coordinates'].str.split(', ', expand=True)

# Converting latitude as well as longitude into the same format (degree with decimal points)
data['latitude'] = data['latitude'].apply(convert_coordinate)
data['longitude'] = data['longitude'].apply(convert_coordinate)

# Printing the first 3 rows after above operations
data.head(3)

,name,coordinates,height,date_added,metadata,latitude,longitude
0,Exatel HQ,"52.2322, 21.0083",0.115,2024-05-06T09:00:00,sector=telecommunications;status=headquarters,0.0,0.0
1,Wrocław IT Center,"51.1079, 17.0385",0.142,2024-05-06T09:15:00,sector=IT services;status=data center,0.0,0.0
2,Kraków Cyber Hub,"50.0647, 19.9450",0.206,2024-05-06T09:30:00,sector=cybersecurity;status=training facility,0.0,0.0


## Inserting Data

Iterating through data (`pandas.DataFrame`) using for loop and leveraging the `.iterrows()` method that returns the generator through which we can iterate. `row` variable is a `pandas.Series` object, so we are able to easily manipulate it.

In [42]:
# For each row insert it into database
for index, row in data.iterrows():
    # Destructurizing columns from the row
    name, latitude, longitude, height, date_added, metadata = row
    
    # Creating insert query with parameterized inputs
    # This approach avoids syntax errors and protects against SQL injection attacks
    add_row_query = f'''
    INSERT INTO points (id, name, lattitude, longitude, height, date_added, metadata) 
    VALUES (?, ?, ?, ?, ?, ?)
    '''
    
    # Data to insert
    data_to_insert = (index, name, latitude, longitude, height, date_added, metadata)
    
    # Executing insert operation
    conn.execute(add_row_query, data_to_insert)
    
    # Commiting changes
    conn.commit()

ValueError: too many values to unpack (expected 6)

## See Loaded Data

Check if data loaded successfully to the database and use `.fetchall()` coords to see that.

In [ ]:
# Execute query that selects all rows from points table
cursor.execute('SELECT * FROM points')

# Get all rows
rows = cursor.fetchall()

# Show only the first 2 rows
rows[:2]

[(0,
  'Exatel HQ',
  '52.2322, 21.0083',
  0.115,
  '2024-05-06T09:00:00',
  'sector=telecommunications;status=headquarters'),
 (1,
  'Wrocław IT Center',
  '51.1079, 17.0385',
  0.142,
  '2024-05-06T09:15:00',
  'sector=IT services;status=data center')]

In [ ]:
perimeter = calculate_perimeter(data)



# map = folium.Map(location=[0, 0], zoom_start=2)
# for index, row in data.iterrows():
#     folium.Marker([row['Lat'], row['Lon']], popup=row['Nazwa']).add_to(map)
# map.save('map.html')